# Coursework Applied Advanced Optimisation

---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London

**Module:** ELEC70066 - Applied Advanced Optimisation

**Version:** 1.1.1 - 29/01/2025

---
---

This coursework covers some of the so-called "geometric problems". These problems belong to an additional family of applications which complements Chapter $1$ (Approximation and Fitting) and Chapter $2$ (Statistical Estimation). Geometric problems cover many important applications, the most famous of which is machine learning. In fact, the popular Support Vector Machines are a family of classification methods which are nothing else than convex optimisation problems.

# Important instructions

You must read this section in its entirety before starting the coursework.

This coursework is intended to be an occasion for you to explore an optimisation topic in autonomy. The coursework is based on "Section $8.6$ Classification" of the reference book $[1]$ (link available on Blackboard). You are supposed to read that section to be able to complete the coursework.

You cannot use the sklearn library, nor any other library in which SVM or other classifiers are already implemented. The objective of the coursework is that you learn how these classifiers are obtained from scratch. This means that you will have to use your knowledge of Convex Optimisation and CVX to solve the questions directly.

You must copy this notebook to your drive; work on the file and submit the completed .ipynb file to Blackboard. Before submitting, make sure that you run all the code cells and save the notebook. This will ensure that the notebook looks exactly how you intend it to look. Obviously it is expected that your submission will have plenty of text cells in which you explain your work.

The coursework consists of 8 questions. Create a new section for each question.

Generally speaking, no help will be provided on the coursework besides clarifications on the wording of the text. You need to figure out by yourself how to solve the coursework. Note that after reading Section $8.6$ of the book, you will have all the required information to be able to solve the coursework.

The coursework is an individual piece of assessment. You can discuss it with other students, but you cannot develop solutions together and you cannot share code, text or figures. You cannot ask other people (including online forums) to solve any part of the coursework on your behalf. Standard plagiarisms policy will be applied.

The use of generative AI tools is not allowed.

# The problem and data

In pattern recognition and classification problems we are given two sets of points in $\mathbb{R}^n$, $\{x_1,\dots,x_N\}$ and $\{y_1,\dots,y_M\}$, and wish to find a function $f : \mathbb{R}^n \to \mathbb{R}$ (within a given family of functions) that is positive on the first set and negative on
the second, i.e.,

$$
f(x_i) > 0,\quad i = 1,\dots,N, \qquad f(y_i) < 0,\quad i = 1, \dots ,M.
$$

If these inequalities hold, we say that $f$ separates, classifies, or discriminates the two sets of points. This is an instance of supervised learning.

In the next subsections you will find $4$ datasets which will be used in the questions below. For datasets $1$, $2$ and $3$ you are given a "train set" and a "test set". You are supposed to develop your optimisation algorithm only on the train set. Do not use the test set for training/optimisation. The purpose of the test set is for you to test a posteriori how the classifier that you have obtained performs. For data set $4$, use the entire set in the optimisation (i.e. there is no test phase).

### Dataset 1

In [ ]:
# Dataset 1. DO NOT CHANGE THE NUMBERS
import numpy as np
X_train_1 = np.array([[ 1.75038375,  1.36985822,  0.73076475,  0.73666559,  0.34224149,
         1.09826685,  0.86211442,  1.24674778,  0.12887349,  0.35909321,
         0.35246291,  0.63681688,  0.54697422,  0.76350366,  1.5921626 ,
         0.51972449,  1.09117696,  1.42626192,  1.06227916,  0.21480643,
         1.42300815,  0.84256355,  1.7533684 ,  0.69974175,  0.76946708,
        -0.18379672,  1.0485532 ,  0.96472069,  1.31625354,  0.36578051,
         0.77772201,  1.64143191],
       [-0.16275141, -0.82300504, -1.30848519, -0.99270645, -1.81370696,
         2.8696898 , -1.06625993, -0.73442029,  0.50866809, -0.99408544,
         0.40021373, -0.10562573,  0.30068813, -0.18723889,  1.5299066 ,
         0.23149303,  0.21033337, -0.37766466,  1.7832445 ,  0.77048214,
         2.38601163, -0.12591569,  0.58083659, -0.21862494,  1.21901444,
        -0.70226941,  1.82745304,  1.02706123, -1.21456594, -0.04151123,
         0.57038679,  0.37708978]]).T

Y_train_1 = np.array([[-1.61978158, -1.42158013, -1.10032926, -0.84133058, -1.61746142,
        -1.37001245, -0.81530917, -2.19997218, -1.17612194,  0.28991103,
        -1.15223907, -0.20748713, -0.26374853, -1.38338996, -0.60422153,
        -0.6807668 , -0.49697651, -1.76927614, -1.07011443, -2.13637377,
        -0.97815482, -1.38615705,  0.16678405, -0.20236665, -0.52664181,
        -0.52065308, -1.06395708, -0.09375681],
       [ 1.29534095,  0.70877882,  0.22482794, -1.4081337 , -0.63963762,
         0.70944757, -0.21818498, -0.87909018,  0.23087154,  1.26274235,
         0.23315878, -1.08721924,  0.46586729, -0.64121136, -0.844737  ,
        -0.18690388, -0.71034608, -0.98534397,  0.18144543, -0.8120575 ,
         0.00244075, -0.28833748,  1.3720753 ,  0.34245524,  1.1515264 ,
         1.11200279,  0.38977343,  0.3943303 ]]).T

X_test_1 = np.array([[ 1.21290648,  0.60647149,  1.21197796,  1.54724686,  1.00972903,
         0.56216029,  0.72973515,  0.39284694,  0.17336897,  0.74244925,
         0.6168276 ,  0.91094238,  0.75389832,  1.22221484,  0.60651985,
         0.54111179,  0.91559423,  0.25201646],
       [-0.71869421, -1.12772433, -0.58861397, -0.0145585 , -1.45496704,
        -1.37186247,  0.35988092,  1.00862714,  0.30279016,  2.55054793,
         2.28438069, -0.35502465, -0.9418674 ,  0.89477983, -1.14724826,
        -0.7799858 ,  1.06303296, -0.55712765]]).T

Y_test_1 = np.array([[-1.19236589, -1.10995069,  0.37669071, -0.95531888, -0.89232474,
        -0.02095101, -0.29050649, -0.37881187, -0.99739798, -1.20639358,
        -0.54710686, -0.25815146, -1.136852  , -0.99897624, -0.78183458,
         0.21608231, -1.39421902, -0.85569085, -0.25452413, -2.08591412,
        -0.7539892 , -0.49801935],
       [-1.09011136,  1.87167148, -1.63772617, -0.20106049, -1.30309817,
         1.05833253, -0.04721317, -1.6502976 , -0.36833975, -1.2191319 ,
        -0.75123888,  0.43614121, -0.09030079, -1.32835851, -0.10259796,
        -0.04046664,  0.06085472,  0.119087  ,  0.81613417, -0.55277698,
        -1.03559666,  0.55011099]]).T

### Dataset 2

In [ ]:
# Dataset 2. DO NOT CHANGE THE NUMBERS
import numpy as np
X_train_2 = np.array([[-1.614901  ,  0.28714288, -1.58856523, -0.62398345, -1.24054186,
         1.3540726 , -1.8376991 , -0.76542848,  1.01551298,  1.56603302,
        -1.03332383, -0.65191468,  0.08107311,  1.1886501 ,  1.55395519,
         1.85121256, -0.38917117,  1.60296587, -1.29848704,  0.8441058 ,
         1.51105349,  1.30539824, -1.78357771, -0.07797286, -1.94890625,
        -0.67010608,  1.26205438, -0.64744794,  1.49213191, -1.32792377,
         0.40606432,  1.1654875 , -1.39691976, -1.53382165],
       [-0.74107291,  2.0226272 , -0.80311089, -1.93652965,  0.93944489,
        -0.76972033,  0.4438167 , -1.28950516, -0.54197248, -0.68859753,
         0.84952455, -1.46468159,  0.89980228, -0.73998096, -1.24199011,
        -0.004975  ,  1.22250794,  0.6042052 , -0.09834692,  1.90774378,
         1.21666545, -1.52465769,  1.46776645, -1.48563138,  0.92646412,
        -2.35182561,  1.08872015,  1.50066709,  1.54360638,  0.08300261,
         1.46244545,  0.42036718, -1.16291727,  1.21058973]]).T

Y_train_2 = np.array([[ 0.33752225,  0.26007797, -0.77360943, -0.80707957, -1.02059109,
         1.21402308, -1.05628487, -0.44228055, -0.40975282, -0.58552875,
         0.73242037,  0.60985187, -0.38448676, -1.10713613,  0.71019777,
         0.42445554,  0.54032669,  0.16261397, -0.7642942 , -0.0826362 ,
        -0.01755153, -1.33428073, -0.05495925,  0.971422  , -0.11024436,
         0.46940181],
       [ 0.1972156 , -0.40692781, -0.21251355, -0.38086556,  0.55460174,
        -0.00750315, -0.08248651,  0.79141975, -0.76573106,  0.28460164,
         0.75396732,  0.23217386,  0.97141896, -0.63721127,  0.41754396,
        -1.47865265, -0.11135927, -0.18183213, -0.03324034,  0.56319091,
        -0.84912994,  0.28134522,  0.52742495, -0.26947631,  0.44022805,
         0.77060324]]).T

X_test_2 = np.array([[-1.18919052,  1.86146486,  0.73290109, -1.02598963, -1.94986405,
         1.98413584, -0.22543794,  1.6738425 , -0.54083103,  0.23855336,
         0.29067775,  0.90452835, -0.7807143 ,  0.92563898,  0.5891893 ,
        -1.8506076 ],
       [ 1.01734226,  1.22983793, -1.32784027, -1.25200345, -1.11164531,
         0.11500191,  1.37788957, -0.04001507, -1.45583732,  1.64623486,
        -1.5711447 , -1.27852394,  1.32218203,  1.77824856, -1.83015366,
        -1.32219896]]).T

Y_test_2 = np.array([[ 0.44030401, -0.96934769,  0.83688707,  0.42107823, -0.64710246,
        -0.80688878, -0.17615909, -0.07870086,  0.37287753,  1.24256423,
        -0.07644516,  0.91728516,  1.4185267 , -0.57032662,  0.32910968,
        -0.62365553,  0.04200134, -0.05018494, -0.25026012,  0.30984256,
         0.13336253,  0.92133881, -0.76288057,  0.45065785],
       [ 0.9436088 , -0.72879255, -0.54252464, -0.95075636, -0.13248588,
        -0.59037888,  1.37253857,  0.75478021, -0.65673396, -0.31060961,
         0.37673711,  0.94192255, -0.24612582,  0.90276183, -0.88489523,
         0.05975249, -1.13783772, -0.72226809,  0.80242032,  0.47174672,
        -0.8985866 ,  0.59402413, -0.07023891, -0.97869024]]).T

### Dataset 3

In [ ]:
# Dataset 3. DO NOT CHANGE THE NUMBERS
import numpy as np
X_train_3 = np.array([[ 0.30920485,  0.74779523, -1.19805934, -1.4065875 ,  0.81773837,
        -1.30667592, -0.22120141,  0.07813527, -0.42443599,  0.03138162,
        -2.02716273, -0.20216417, -1.77093289,  0.19895122,  0.27524467,
        -0.9843448 , -0.31251963, -1.72018154, -2.05161935, -0.81445041,
         0.75566503, -1.61181827,  0.24942261,  0.43094924,  0.26285439,
        -1.49128776, -1.2378428 ,  0.15350841, -1.63601617, -0.03734756,
        -0.22413227],
       [-0.127041  ,  0.54957438,  1.40047603, -1.03687185,  1.64300265,
        -0.60711187,  1.00868763,  0.18433247,  1.30854456,  1.27867747,
        -0.07552655,  0.09829492, -0.15602456,  2.27694191,  0.58976897,
         1.29546509,  1.85361272, -0.11323539,  0.3295589 ,  0.03881959,
         0.62671139, -1.16122353,  0.37041953, -0.51610516,  0.08970487,
         0.68265883,  0.7264372 ,  0.8912278 ,  1.14409337,  0.53962298,
         0.94477311]]).T

Y_train_3 = np.array([[-0.48691402,  0.50954902,  1.35242892,  1.89600064, -0.33075371,
        -0.62663858, -0.85151619, -0.59739923,  0.15581821,  0.24130156,
         0.0081454 ,  1.01975478,  1.22467352,  0.51038341,  0.88796385,
         0.15940018, -0.1636123 ,  1.53916295, -0.6851801 , -0.42081847,
        -0.36004546,  1.10877664,  1.82124821, -0.04156698,  0.23092986,
         1.85584796, -0.34194837,  0.89063568, -0.99471418],
       [ 0.57259859, -1.66427178,  0.60859541, -0.42653506, -0.55325533,
        -0.45097084, -0.06022663,  0.27829883, -1.14573741, -1.74371889,
        -0.58656254, -0.39964896, -1.45537741, -1.09925804, -1.22007405,
        -1.05506347, -1.53782127, -0.40646646,  0.56462893, -1.03126372,
         0.03132026, -0.56596703, -0.46607732, -0.71850484, -1.16563865,
         0.39711336, -0.50298565, -0.73843955, -0.7858591 ]]).T

X_test_3 = np.array([[-1.55182484, -1.14359579, -0.28724359,  0.4860189 ,  0.59348201,
         0.20764271, -0.71978547, -0.52880272, -0.83190068, -0.17344159,
         0.11892081, -1.55268459, -0.97786341,  1.15004209, -1.55962329,
        -0.98048598, -0.73153577, -0.75993791, -0.22465612],
       [ 0.46910804,  0.76280671,  1.0525906 ,  1.96041925,  0.25278868,
         0.88856402,  1.72334673,  1.28542054,  0.93244079,  1.50362044,
         0.14210094, -0.3087488 ,  1.51041733, -0.46423271, -0.26480295,
         0.8478617 ,  1.24236135,  1.31112526,  1.70271788]]).T

Y_test_3 = np.array([[ 0.59260113, -0.87894137, -0.02974639,  1.7173351 , -0.09130345,
         1.31650323,  0.3712312 ,  1.39730283,  1.12756772,  0.62682189,
        -0.34408365,  1.33380816,  1.89457786, -0.31784745,  1.37086214,
         1.69531142, -0.53631325,  1.60097254,  0.98840214,  1.81872078,
         0.67050908],
       [-1.44842808,  0.65349306, -0.63475756, -0.17875463, -0.93824346,
        -1.10788814, -1.30192489,  0.39040035, -1.65289561, -2.14337167,
        -0.11501714, -0.73018114, -1.57392425, -1.493817  ,  0.22375044,
        -0.91174565, -0.20549059, -0.09531556, -1.75933045,  0.7346544 ,
        -1.01221609]]).T

### Dataset 4

In [ ]:
# Dataset 4. DO NOT CHANGE THE NUMBERS
import numpy as np
X_4 = np.array([[ 3.496052,  4.042794,  2.79643 ,  5.085416,  4.240754,  3.737034,
         3.343466,  3.786328,  4.965464,  3.826312,  4.208512,  4.620014,
         3.242036,  5.14892 ,  2.50782 ,  3.528098,  4.719288,  5.184396,
         5.086984,  3.900596],
       [-2.938138,  0.507444,  2.100728, -0.066346,  0.308112,  1.280272,
         3.851106,  0.200998,  0.104566, -1.468236, -2.356998,  2.863952,
         1.513512, -2.811326,  1.007538,  1.21716 ,  1.255772,  1.17943 ,
        -2.121112, -0.276458]]).T

Y_4 = np.array([[-4.47517 , -3.616592, -3.222338, -1.616118, -5.363638, -3.54466 ,
        -1.163848, -1.032822, -1.36367 , -1.990576, -1.959902, -0.24304 ,
        -1.288602, -0.81389 , -1.89679 , -1.068004, -2.55192 , -4.272996,
        -1.77429 ,  0.303408],
       [ 2.363466,  4.178916,  2.78908 ,  0.5145  ,  1.867194,  2.923438,
         4.613742,  0.950796,  0.377692,  1.884344,  1.461572, -0.978432,
         3.36434 ,  2.866108,  3.021634,  1.55918 ,  1.496068,  1.593088,
         2.453626,  1.409632]]).T

Z_4 = np.array([[ 1.710198,  2.58181 ,  0.581826, -2.765266,  2.969792,  1.069866,
        -1.743714,  1.217356,  2.143554, -2.254784, -3.259284,  2.706466,
         0.898268,  0.058898, -2.59896 , -3.25409 ,  4.040442, -3.34033 ,
        -3.096212, -0.716478],
       [-3.13698 , -4.892258, -3.686858, -4.64716 , -4.04887 , -3.83376 ,
        -4.468408, -4.850902, -3.36238 , -4.180288, -6.078254, -4.428228,
        -3.690582, -4.903822, -4.286912, -4.954782, -3.865414, -3.960376,
        -5.242314, -2.98557 ]]).T

# Questions

0.   Read Section $8.6$ of $[1]$.
1.   For dataset $1$, determine a standard support vector classifier with $\gamma = 0.1$ (notation from Section $8.6$ of the book). Plot the points, the classifier $f$ and the associated slab. **[10 marks]**

2.   Use the test set to assess how good the classifier is on unseen data. To this end, first define a "metric" (or more than one if you prefer) which numerically quantifies how good or bad a classifier is. Explain why you picked this metric and how the metric works. **[10 marks]**

3.   Discuss, possibly with the help of figures, the role of the parameter $\gamma$ and find for which $\gamma$ (or $\gamma$'s) your classifier behaves best on the test set. **[10 marks]**

4.   Now determine a linear classifier by approximating it via logistic modeling. Compare and discuss the results with those obtained in parts 1. 2. and 3. **[10 marks]**

5.   For dataset $2$, determine a standard support vector classifier with $\gamma=0.1$. Plot the points, the classifier $f$ and the associated slab. Score your classifier on the test set using your metric. **[5 marks]**

6.   Implement nonlinear classifiers for dataset $2$. There are many options here. Discuss as much as you can and try to find the best classifier among those which are obtainable by means of convex optimisation. The more details you provide to explain how you reached your final choice, the better. **[25 marks]**

7.   Repeat parts 5. and 6. on dataset 3. **[15 marks]**

8.   Dataset $4$ consists of three sets of points corresponding to three classes instead of two. Explain how you would go about finding a classifier for this case. Try to find a classifier and plot the points and the classifiers. Describe your results. **[15 marks]**

# Answers

*   Name: Amor
*   Surname: Zhao 
*   CID: 02019680